# SwellSight Real-to-Synthetic Pipeline - Setup and Installation

This notebook handles environment preparation and dependency management for the SwellSight real-to-synthetic image generation pipeline.

## Overview
Based on the `real_to_synthetic_pipeline.py` script, this notebook provides:
- Environment detection and GPU configuration
- Installation of MiDaS depth estimation dependencies
- ControlNet and diffusion model setup
- SwellSight package installation and configuration
- Directory structure creation for pipeline data
- Configuration management for reproducible results

## Pipeline Components
1. **MiDaS Depth Extraction**: Extract depth maps from real beach images
2. **Data Augmentation**: Generate parameter variations for synthetic diversity
3. **ControlNet Generation**: Create synthetic images using depth conditioning

## Prerequisites
- Python 3.8+ environment
- CUDA-capable GPU (recommended for performance)
- At least 8GB GPU memory for ControlNet models
- Real beach images in `data/real/images/` directory

---

## 1. Environment Detection and Path Configuration

In [1]:
import sys
from pathlib import Path

# Check if running in Google Colab
if 'google.colab' in sys.modules:
    from google.colab import drive
    print("Mounting Google Drive...")

    try:
        # Attempt 1: Standard mount
        drive.mount('/content/drive')
        print("✓ Google Drive mounted successfully")
    except Exception as e:
        print(f"Standard mount failed: {e}")

        # Attempt 2: Force remount with extended timeout (robust fallback)
        print("Trying force remount with extended timeout...")
        try:
            drive.mount('/content/drive', force_remount=True, timeout_ms=300000)
            print("✓ Force remount successful")
        except Exception as e2:
            print(f"❌ Critical failure mounting drive: {e2}")
            raise

    # Verify the specific project path exists
    # Adjust this path if your folder structure changes
    PROJECT_PATH = Path('/content/drive/MyDrive/SwellSight')
    if PROJECT_PATH.exists():
        print(f"✓ Project directory found: {PROJECT_PATH}")
    else:
        print(f"⚠️ Project directory not found at: {PROJECT_PATH}")
else:
    print("Not running in Google Colab. Skipping Drive mount.")

Mounting Google Drive...
Mounted at /content/drive
✓ Google Drive mounted successfully
✓ Project directory found: /content/drive/MyDrive/SwellSight


In [2]:
import sys
import os
import platform
from pathlib import Path
import subprocess
import json
from datetime import datetime
import logging

# Detect environment
IN_COLAB = 'google.colab' in sys.modules
IS_KAGGLE = 'kaggle_secrets' in sys.modules
IS_LOCAL = not (IN_COLAB or IS_KAGGLE)

print(f"Environment Detection:")
print(f"  Google Colab: {IN_COLAB}")
print(f"  Kaggle: {IS_KAGGLE}")
print(f"  Local Jupyter: {IS_LOCAL}")
print(f"  Python version: {sys.version}")
print(f"  Platform: {platform.platform()}")

# Set base paths based on environment
if IN_COLAB:
    BASE_PATH = Path('/content')
    DRIVE_PATH = Path('/content/drive/MyDrive/SwellSight')
elif IS_KAGGLE:
    BASE_PATH = Path('/kaggle/working')
    DRIVE_PATH = Path('/kaggle/working/SwellSight')
else:
    BASE_PATH = Path.cwd()
    DRIVE_PATH = BASE_PATH / 'SwellSight'

print(f"\nPath Configuration:")
print(f"  Base path: {BASE_PATH}")
print(f"  Drive path: {DRIVE_PATH}")

# Configure logging for pipeline
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

Environment Detection:
  Google Colab: True
  Kaggle: False
  Local Jupyter: False
  Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
  Platform: Linux-6.6.105+-x86_64-with-glibc2.35

Path Configuration:
  Base path: /content
  Drive path: /content/drive/MyDrive/SwellSight


## 2. GPU Detection and Configuration

In [3]:
# GPU Detection
try:
    import torch
    gpu_available = torch.cuda.is_available()
    gpu_count = torch.cuda.device_count()

    print(f"GPU Configuration:")
    print(f"  CUDA available: {gpu_available}")
    print(f"  GPU count: {gpu_count}")

    if gpu_available:
        for i in range(gpu_count):
            gpu_name = torch.cuda.get_device_name(i)
            gpu_memory = torch.cuda.get_device_properties(i).total_memory / 1024**3
            print(f"  GPU {i}: {gpu_name} ({gpu_memory:.1f} GB)")

        # Set default device
        device = torch.device('cuda:0')
        print(f"\nUsing device: {device}")

        # Clear GPU cache
        torch.cuda.empty_cache()
        print("GPU cache cleared")

        # Memory recommendations
        current_gpu = torch.cuda.current_device()
        gpu_memory = torch.cuda.get_device_properties(current_gpu).total_memory / 1e9

        if gpu_memory < 6:
            print("⚠️  Warning: Less than 6GB GPU memory. Consider using smaller models.")
        elif gpu_memory >= 12:
            print("✅ Excellent: Sufficient memory for full pipeline with large models.")
        else:
            print("✅ Good: Sufficient memory for pipeline with standard models.")
    else:
        device = torch.device('cpu')
        print(f"\nUsing device: {device} (GPU not available)")
        print("⚠️  No GPU detected. Pipeline will run on CPU (much slower).")

    # Store device for later use
    DEVICE = device

except ImportError:
    print("PyTorch not installed yet - will check GPU after installation.")
    gpu_available = False
    gpu_count = 0
    DEVICE = None

GPU Configuration:
  CUDA available: True
  GPU count: 1
  GPU 0: Tesla T4 (14.7 GB)

Using device: cuda:0
GPU cache cleared
✅ Excellent: Sufficient memory for full pipeline with large models.


## 3. Google Drive Mounting (Colab Only)

In [4]:
if IN_COLAB:
    from google.colab import drive
    import time

    print("Mounting Google Drive...")

    # Try multiple mount strategies
    mount_success = False

    # Strategy 1: Standard mount
    try:
        drive.mount('/content/drive')
        mount_success = True
        print("✓ Standard mount successful")
    except Exception as e:
        print(f"Standard mount failed: {e}")

        # Strategy 2: Force remount with extended timeout
        try:
            print("Trying force remount with extended timeout...")
            drive.mount('/content/drive', force_remount=True, timeout_ms=300000)
            mount_success = True
            print("✓ Force remount successful")
        except Exception as e2:
            print(f"Force remount failed: {e2}")

            # Strategy 3: Wait and retry
            try:
                print("Waiting 10 seconds and retrying...")
                time.sleep(10)
                drive.mount('/content/drive', force_remount=True)
                mount_success = True
                print("✓ Retry mount successful")
            except Exception as e3:
                print(f"All mount strategies failed: {e3}")
                print("Continuing with local storage...")
                mount_success = False

    # Verify drive access or create local fallback
    if mount_success and Path('/content/drive/MyDrive').exists():
        print("✓ Google Drive mounted successfully")
    else:
        print("⚠ Using local storage instead of Google Drive")
        DRIVE_PATH = BASE_PATH / 'SwellSight'
        print(f"✓ Local directory structure will be created: {DRIVE_PATH}")
else:
    print("Not running in Google Colab. Using local storage.")

Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ Standard mount successful
✓ Google Drive mounted successfully


## 4. Package Installation

In [5]:
# Core ML and Computer Vision packages
packages_to_install = [
    # Core ML frameworks
    'torch>=2.0.0',
    'torchvision>=0.15.0',
    'torchaudio>=2.0.0',
    'transformers>=4.30.0',
    'diffusers>=0.20.0',
    'accelerate>=0.20.0',

    # Computer Vision
    'opencv-python>=4.8.0',
    'Pillow>=9.5.0',
    'scikit-image>=0.20.0',
    'controlnet-aux>=0.0.6',

    # MiDaS dependencies
    'timm>=0.9.0',
    'einops>=0.6.0',
    'omegaconf>=2.3.0',

    # ControlNet dependencies
    'xformers>=0.0.20',
    'invisible-watermark>=0.2.0',
    'safetensors>=0.3.0',

    # Data Science and Analysis
    'numpy>=1.24.0',
    'pandas>=2.0.0',
    'matplotlib>=3.7.0',
    'seaborn>=0.12.0',
    'plotly>=5.14.0',
    'scipy>=1.10.0',
    'scikit-learn>=1.3.0',

    # Utilities
    'tqdm>=4.65.0',
    'ipywidgets>=8.0.0'
]

print("Installing required packages...")
print("This may take several minutes...\n")

for package in packages_to_install:
    try:
        print(f"Installing {package}...")
        result = subprocess.run(
            [sys.executable, '-m', 'pip', 'install', package, '--quiet'],
            capture_output=True,
            text=True
        )
        if result.returncode == 0:
            print(f"✓ {package.split('>=')[0]} installed successfully")
        else:
            print(f"⚠️  Optional package {package} failed: {result.stderr}")
    except Exception as e:
        print(f"⚠️  Error installing {package}: {e}")

print("\n✓ Package installation completed")

Installing required packages...
This may take several minutes...

Installing torch>=2.0.0...
✓ torch installed successfully
Installing torchvision>=0.15.0...
✓ torchvision installed successfully
Installing torchaudio>=2.0.0...
✓ torchaudio installed successfully
Installing transformers>=4.30.0...
✓ transformers installed successfully
Installing diffusers>=0.20.0...
✓ diffusers installed successfully
Installing accelerate>=0.20.0...
✓ accelerate installed successfully
Installing opencv-python>=4.8.0...
✓ opencv-python installed successfully
Installing Pillow>=9.5.0...
✓ Pillow installed successfully
Installing scikit-image>=0.20.0...
✓ scikit-image installed successfully
Installing controlnet-aux>=0.0.6...
✓ controlnet-aux installed successfully
Installing timm>=0.9.0...
✓ timm installed successfully
Installing einops>=0.6.0...
✓ einops installed successfully
Installing omegaconf>=2.3.0...
✓ omegaconf installed successfully
Installing xformers>=0.0.20...
✓ xformers installed successfull

## 5. Import Core Libraries and Verify Installation

In [6]:
# Import and verify core libraries
import warnings
warnings.filterwarnings('ignore')

try:
    # Core ML libraries
    import torch
    import torchvision
    from transformers import DPTImageProcessor, DPTForDepthEstimation
    print(f"✓ PyTorch: {torch.__version__}")
    print(f"✓ Torchvision: {torchvision.__version__}")
    print(f"✓ Transformers: Available")

    # Try importing diffusers for ControlNet
    try:
        from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
        print(f"✓ Diffusers: Available (ControlNet support)")
        CONTROLNET_AVAILABLE = True
    except ImportError:
        print(f"⚠ Diffusers: Not available (will use fallback generation)")
        CONTROLNET_AVAILABLE = False

    # Computer Vision libraries
    import cv2
    from PIL import Image
    import numpy as np
    print(f"✓ OpenCV: {cv2.__version__}")
    print(f"✓ PIL: Available")
    print(f"✓ NumPy: {np.__version__}")

    # Data Science libraries
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    from scipy import stats
    from sklearn.metrics import mean_absolute_error, accuracy_score
    print(f"✓ Pandas: {pd.__version__}")
    print(f"✓ Matplotlib: Available")
    print(f"✓ Seaborn: Available")
    print(f"✓ SciPy: Available")
    print(f"✓ Scikit-learn: Available")

    # Utilities
    from tqdm.auto import tqdm
    import ipywidgets as widgets
    from IPython.display import display, HTML, clear_output
    print(f"✓ TQDM: Available")
    print(f"✓ IPyWidgets: Available")

    print("\n✅ All core libraries imported successfully!")

    # Update GPU detection with imported torch
    if DEVICE is None:
        gpu_available = torch.cuda.is_available()
        gpu_count = torch.cuda.device_count()
        DEVICE = torch.device('cuda:0' if gpu_available else 'cpu')
        print(f"\n🚀 GPU Status: {DEVICE}")

except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please run the package installation cell again.")
    CONTROLNET_AVAILABLE = False

✓ PyTorch: 2.9.0+cu126
✓ Torchvision: 0.24.0+cu126
✓ Transformers: Available


Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


✓ Diffusers: Available (ControlNet support)
✓ OpenCV: 4.12.0
✓ PIL: Available
✓ NumPy: 2.0.2
✓ Pandas: 2.2.2
✓ Matplotlib: Available
✓ Seaborn: Available
✓ SciPy: Available
✓ Scikit-learn: Available
✓ TQDM: Available
✓ IPyWidgets: Available

✅ All core libraries imported successfully!


## 6. Directory Structure Creation

In [7]:
# Create directory structure for pipeline data
print("Creating pipeline directory structure...")

# Base directories from real_to_synthetic_pipeline.py
base_dirs = {
    'data': 'Main data directory',
    'data/real': 'Real beach camera images',
    'data/real/images': 'Input real images',
    'data/depth_maps': 'MiDaS extracted depth maps',
    'data/synthetic': 'Generated synthetic images',
    'data/synthetic/images': 'Synthetic image outputs',
    'data/synthetic/labels': 'Synthetic image metadata',
    'data/metadata': 'Pipeline metadata and configs',
    'checkpoints': 'Model checkpoints and saved states',
    'logs': 'Pipeline execution logs',
    'results': 'Analysis results and reports',
    'models': 'Downloaded model files'
}

created_dirs = []
existing_dirs = []

# Change to DRIVE_PATH for directory creation
os.chdir(DRIVE_PATH.parent)
DRIVE_PATH.mkdir(exist_ok=True)
os.chdir(DRIVE_PATH)

for dir_path, description in base_dirs.items():
    full_path = Path(dir_path)
    if full_path.exists():
        existing_dirs.append((dir_path, description))
    else:
        full_path.mkdir(parents=True, exist_ok=True)
        created_dirs.append((dir_path, description))

print(f"\n📁 Created {len(created_dirs)} new directories:")
for dir_path, description in created_dirs:
    print(f"   ✅ {dir_path:<25} - {description}")

if existing_dirs:
    print(f"\n📂 Found {len(existing_dirs)} existing directories:")
    for dir_path, description in existing_dirs:
        print(f"   📁 {dir_path:<25} - {description}")

# Create .gitkeep files for empty directories
empty_dirs = ['data/real/images', 'checkpoints', 'logs']
for dir_path in empty_dirs:
    gitkeep_path = Path(dir_path) / '.gitkeep'
    if not gitkeep_path.exists():
        gitkeep_path.touch()

print("\n🏗️  Directory structure created successfully!")

Creating pipeline directory structure...

📁 Created 0 new directories:

📂 Found 12 existing directories:
   📁 data                      - Main data directory
   📁 data/real                 - Real beach camera images
   📁 data/real/images          - Input real images
   📁 data/depth_maps           - MiDaS extracted depth maps
   📁 data/synthetic            - Generated synthetic images
   📁 data/synthetic/images     - Synthetic image outputs
   📁 data/synthetic/labels     - Synthetic image metadata
   📁 data/metadata             - Pipeline metadata and configs
   📁 checkpoints               - Model checkpoints and saved states
   📁 logs                      - Pipeline execution logs
   📁 results                   - Analysis results and reports
   📁 models                    - Downloaded model files

🏗️  Directory structure created successfully!


## 7. Global Configuration Setup

In [8]:
# Global configuration dictionary
CONFIG = {
    # Environment settings
    'environment': {
        'platform': 'colab' if IN_COLAB else 'kaggle' if IS_KAGGLE else 'local',
        'device': str(DEVICE),
        'gpu_available': gpu_available,
        'gpu_count': gpu_count if gpu_available else 0,
        'controlnet_available': CONTROLNET_AVAILABLE
    },

    # Path configuration
    'paths': {
        'base_path': str(BASE_PATH),
        'drive_path': str(DRIVE_PATH),
        'data_path': str(DRIVE_PATH / 'data'),
        'real_images_path': str(DRIVE_PATH / 'data' / 'real' / 'images'),
        'depth_maps_path': str(DRIVE_PATH / 'data' / 'depth_maps'),
        'synthetic_data_path': str(DRIVE_PATH / 'data' / 'synthetic'),
        'metadata_path': str(DRIVE_PATH / 'data' / 'metadata'),
        'models_path': str(DRIVE_PATH / 'models'),
        'checkpoints_path': str(DRIVE_PATH / 'checkpoints'),
        'results_path': str(DRIVE_PATH / 'results'),
        'logs_path': str(DRIVE_PATH / 'logs')
    },

    # Model configuration
    'models': {
        'midas': {
            'default': 'Intel/dpt-large',
            'alternatives': ['Intel/dpt-hybrid-midas', 'Intel/dpt-base'],
            'description': 'MiDaS depth estimation models'
        },
        'controlnet': {
            'default': 'lllyasviel/sd-controlnet-depth',
            'description': 'ControlNet depth-conditioned generation'
        },
        'stable_diffusion_model': 'runwayml/stable-diffusion-v1-5'
    },

    # Processing parameters
    'processing': {
        'max_images': 800,
        'synthetic_per_real': 3,
        'batch_size': 1,
        'image_size': [768, 768],
        'max_images_per_session': 500,
        'checkpoint_interval': 10,
        'quality_threshold': 0.3,
        'image_extensions': ['jpg', 'jpeg', 'png', 'bmp']
    },

    # Device configuration
    'device': {
        'auto_detect': True,
        'preferred': 'cuda',
        'fallback': 'cpu'
    },

    # Augmentation parameters
    'augmentation': {
        'enabled': True,
        'parameters': {
            'guidance_scale': 7.5,
            'num_inference_steps': 20,
            'controlnet_conditioning_scale': 1.0
        }
    },

    # Quality thresholds
    'quality': {
        'min_overall_quality': 0.5,
        'min_diversity_score': 0.3,
        'min_balance_score': 0.2,
        'drift_threshold': 0.1
    },

    # Session metadata
    'session': {
        'created_at': datetime.now().isoformat(),
        'session_id': f"swellsight_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        'version': '1.0'
    }
}

# Save configuration
config_path = Path('data/metadata/pipeline_config.json')
with open(config_path, 'w') as f:
    json.dump(CONFIG, f, indent=2)

print(f"✅ Configuration saved to: {config_path}")

# Display key settings
print("\n⚙️  Pipeline Configuration Summary:")
print(f"   📊 Max images to process: {CONFIG['processing']['max_images']}")
print(f"   🎨 Synthetic per real image: {CONFIG['processing']['synthetic_per_real']}")
print(f"   🧠 Default MiDaS model: {CONFIG['models']['midas']['default']}")
print(f"   🎯 Default ControlNet: {CONFIG['models']['controlnet']['default']}")
print(f"   📏 Quality threshold: {CONFIG['processing']['quality_threshold']}")
print(f"   Session ID: {CONFIG['session']['session_id']}")
print(f"   Platform: {CONFIG['environment']['platform'].upper()}")
print(f"   Device: {CONFIG['environment']['device']}")

# Store config for other notebooks
PIPELINE_CONFIG = CONFIG
globals()['SWELLSIGHT_CONFIG'] = CONFIG

print("\n🎛️  Configuration setup completed!")

✅ Configuration saved to: data/metadata/pipeline_config.json

⚙️  Pipeline Configuration Summary:
   📊 Max images to process: 800
   🎨 Synthetic per real image: 3
   🧠 Default MiDaS model: Intel/dpt-large
   🎯 Default ControlNet: lllyasviel/sd-controlnet-depth
   📏 Quality threshold: 0.3
   Session ID: swellsight_20260109_162959
   Platform: COLAB
   Device: cuda:0

🎛️  Configuration setup completed!


## 8. SwellSight Package Installation

In [9]:
# Install SwellSight package in development mode
print("Installing SwellSight package...")

# Check if we're in the SwellSight directory
current_dir = Path.cwd()
swellsight_root = None

# Look for setup.py or pyproject.toml
for parent in [current_dir] + list(current_dir.parents):
    if (parent / 'setup.py').exists() or (parent / 'pyproject.toml').exists():
        if (parent / 'swellsight').exists():  # Check for swellsight package
            swellsight_root = parent
            break

if swellsight_root:
    print(f"📁 Found SwellSight root: {swellsight_root}")

    # Install in development mode
    try:
        os.chdir(swellsight_root)
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-e", ".", "--quiet"])
        print("✅ SwellSight package installed in development mode")

        # Test import
        try:
            from swellsight.data.midas_depth_extractor import MiDaSDepthExtractor
            from swellsight.data.controlnet_generator import ControlNetSyntheticGenerator, AugmentationParameterSystem
            print("✅ SwellSight imports successful")

            # Store configuration
            SWELLSIGHT_ROOT = swellsight_root
            print(f"📋 SwellSight root path: {SWELLSIGHT_ROOT}")

        except ImportError as e:
            print(f"⚠️  SwellSight import warning: {e}")
            print("   This may be normal if dependencies are still being set up")

    except subprocess.CalledProcessError as e:
        print(f"❌ SwellSight installation failed: {e}")

else:
    print("⚠️  SwellSight root directory not found")
    print("   Please ensure you're running this notebook from within the SwellSight project")
    SWELLSIGHT_ROOT = current_dir

print("\n📦 SwellSight package setup completed!")

Installing SwellSight package...
⚠️  SwellSight root directory not found
   Please ensure you're running this notebook from within the SwellSight project

📦 SwellSight package setup completed!


## 9. Logging Configuration

In [10]:
# Configure logging
log_file = DRIVE_PATH / 'logs' / f"swellsight_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

# Create logger
logger = logging.getLogger('SwellSight')
logger.setLevel(logging.INFO)

# Clear any existing handlers
logger.handlers.clear()

# Create formatters
formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# File handler
file_handler = logging.FileHandler(log_file)
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Console handler for Jupyter
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

# Test logging
logger.info("SwellSight logging system initialized")
logger.info(f"Session ID: {CONFIG['session']['session_id']}")
logger.info(f"Log file: {log_file}")

print(f"✓ Logging configured")
print(f"  Log file: {log_file}")

# Make logger globally available
globals()['SWELLSIGHT_LOGGER'] = logger

2026-01-09 16:30:00 - SwellSight - INFO - SwellSight logging system initialized
INFO:SwellSight:SwellSight logging system initialized
2026-01-09 16:30:00 - SwellSight - INFO - Session ID: swellsight_20260109_162959
INFO:SwellSight:Session ID: swellsight_20260109_162959
2026-01-09 16:30:00 - SwellSight - INFO - Log file: /content/drive/MyDrive/SwellSight/logs/swellsight_20260109_163000.log
INFO:SwellSight:Log file: /content/drive/MyDrive/SwellSight/logs/swellsight_20260109_163000.log


✓ Logging configured
  Log file: /content/drive/MyDrive/SwellSight/logs/swellsight_20260109_163000.log


## 10. Utility Functions

In [11]:
def save_checkpoint(data, checkpoint_name, description=""):
    """Save checkpoint data to drive."""
    checkpoint_file = DRIVE_PATH / 'checkpoints' / f"{checkpoint_name}.json"

    checkpoint_data = {
        'timestamp': datetime.now().isoformat(),
        'session_id': CONFIG['session']['session_id'],
        'checkpoint_name': checkpoint_name,
        'description': description,
        'data': data
    }

    with open(checkpoint_file, 'w') as f:
        json.dump(checkpoint_data, f, indent=2, default=str)

    logger.info(f"Checkpoint saved: {checkpoint_name} -> {checkpoint_file}")
    return checkpoint_file

def load_checkpoint(checkpoint_name):
    """Load checkpoint data from drive."""
    checkpoint_file = DRIVE_PATH / 'checkpoints' / f"{checkpoint_name}.json"

    if checkpoint_file.exists():
        with open(checkpoint_file, 'r') as f:
            checkpoint_data = json.load(f)

        logger.info(f"Checkpoint loaded: {checkpoint_name} from {checkpoint_file}")
        return checkpoint_data['data']
    else:
        logger.warning(f"Checkpoint not found: {checkpoint_name}")
        return None

def clear_gpu_memory():
    """Clear GPU memory cache."""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        logger.info("GPU memory cache cleared")

def get_memory_usage():
    """Get current memory usage information."""
    import psutil

    # System memory
    memory = psutil.virtual_memory()
    memory_info = {
        'system_memory_total_gb': memory.total / 1024**3,
        'system_memory_used_gb': memory.used / 1024**3,
        'system_memory_percent': memory.percent
    }

    # GPU memory
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        gpu_allocated = torch.cuda.memory_allocated(0) / 1024**3
        gpu_cached = torch.cuda.memory_reserved(0) / 1024**3

        memory_info.update({
            'gpu_memory_total_gb': gpu_memory,
            'gpu_memory_allocated_gb': gpu_allocated,
            'gpu_memory_cached_gb': gpu_cached,
            'gpu_memory_free_gb': gpu_memory - gpu_cached
        })

    return memory_info

def display_progress_widget(description="Processing..."):
    """Create and display a progress widget."""
    progress_bar = widgets.IntProgress(
        value=0,
        min=0,
        max=100,
        description=description,
        bar_style='info',
        style={'bar_color': '#1f77b4'},
        orientation='horizontal'
    )

    status_label = widgets.Label(value="Initializing...")

    display(widgets.VBox([progress_bar, status_label]))

    return progress_bar, status_label

print("✓ Utility functions defined")

# Test checkpoint functionality
test_data = {'setup_completed': True, 'timestamp': datetime.now().isoformat()}
save_checkpoint(test_data, 'setup_test', 'Setup completion test')
loaded_data = load_checkpoint('setup_test')
print(f"✓ Checkpoint system tested: {loaded_data['setup_completed']}")

✓ Utility functions defined


2026-01-09 16:30:00 - SwellSight - INFO - Checkpoint saved: setup_test -> /content/drive/MyDrive/SwellSight/checkpoints/setup_test.json
INFO:SwellSight:Checkpoint saved: setup_test -> /content/drive/MyDrive/SwellSight/checkpoints/setup_test.json
2026-01-09 16:30:00 - SwellSight - INFO - Checkpoint loaded: setup_test from /content/drive/MyDrive/SwellSight/checkpoints/setup_test.json
INFO:SwellSight:Checkpoint loaded: setup_test from /content/drive/MyDrive/SwellSight/checkpoints/setup_test.json


✓ Checkpoint system tested: True


## 11. Final Verification and Summary

In [12]:
# Final verification of setup
print("🔍 Performing final setup verification...\n")

verification_results = {
    "Environment": "✅ Ready",
    "GPU": "❓ Checking...",
    "Core Dependencies": "❓ Checking...",
    "MiDaS": "❓ Checking...",
    "ControlNet": "❓ Checking...",
    "SwellSight Package": "❓ Checking...",
    "Directory Structure": "❓ Checking...",
    "Configuration": "❓ Checking..."
}

# GPU Check
try:
    if torch.cuda.is_available():
        verification_results["GPU"] = f"✅ {torch.cuda.get_device_name()}"
    else:
        verification_results["GPU"] = "⚠️  CPU only"
except:
    verification_results["GPU"] = "❌ PyTorch not available"

# Core Dependencies Check
try:
    import torch, torchvision, transformers, diffusers
    verification_results["Core Dependencies"] = "✅ Installed"
except ImportError as e:
    verification_results["Core Dependencies"] = f"❌ Missing: {e}"

# MiDaS Check
try:
    import timm
    verification_results["MiDaS"] = "✅ Ready"
except ImportError:
    verification_results["MiDaS"] = "❌ Dependencies missing"

# ControlNet Check
try:
    from diffusers import StableDiffusionControlNetPipeline
    verification_results["ControlNet"] = "✅ Ready"
except ImportError:
    verification_results["ControlNet"] = "❌ Diffusers not available"

# SwellSight Package Check
try:
    from swellsight.data.midas_depth_extractor import MiDaSDepthExtractor
    verification_results["SwellSight Package"] = "✅ Installed"
except ImportError:
    verification_results["SwellSight Package"] = "⚠️  Import issues (may be normal)"

# Directory Structure Check
required_dirs = ['data/real/images', 'data/synthetic', 'data/depth_maps']
if all(Path(d).exists() for d in required_dirs):
    verification_results["Directory Structure"] = "✅ Created"
else:
    verification_results["Directory Structure"] = "❌ Missing directories"

# Configuration Check
if Path('data/metadata/pipeline_config.json').exists():
    verification_results["Configuration"] = "✅ Saved"
else:
    verification_results["Configuration"] = "❌ Not saved"

# Display results
print("📋 Setup Verification Results:")
print("=" * 50)
for component, status in verification_results.items():
    print(f"{component:<20}: {status}")

# Overall status
failed_components = [k for k, v in verification_results.items() if v.startswith("❌")]
warning_components = [k for k, v in verification_results.items() if v.startswith("⚠️")]

print("\n" + "=" * 50)
if not failed_components:
    print("🎉 SETUP COMPLETED SUCCESSFULLY!")
    print("\n✅ Ready to proceed to the next notebook:")
    print("   📓 02_Data_Import_and_Preprocessing.ipynb")

    if warning_components:
        print(f"\n⚠️  Note: {len(warning_components)} components have warnings but should work")
else:
    print(f"❌ SETUP INCOMPLETE - {len(failed_components)} components failed")
    print("\n🔧 Please resolve the following issues:")
    for component in failed_components:
        print(f"   - {component}")

print("\n📚 Next Steps:")
print("   1. Place real beach images in data/real/images/")
print("   2. Run 02_Data_Import_and_Preprocessing.ipynb")
print("   3. Execute the full pipeline with 03_MiDaS_Depth_Extraction.ipynb")
print("\n" + "=" * 50)

# Display comprehensive system status
memory_info = get_memory_usage()

print("\n" + "=" * 60)
print("SWELLSIGHT SETUP COMPLETED SUCCESSFULLY")
print("=" * 60)

print(f"\n📋 Session Information:")
print(f"  Session ID: {CONFIG['session']['session_id']}")
print(f"  Platform: {CONFIG['environment']['platform'].upper()}")
print(f"  Created: {CONFIG['session']['created_at']}")

print(f"\n🖥️  Hardware Configuration:")
print(f"  Device: {CONFIG['environment']['device']}")
print(f"  GPU Available: {CONFIG['environment']['gpu_available']}")
if CONFIG['environment']['gpu_available']:
    print(f"  GPU Count: {CONFIG['environment']['gpu_count']}")
    print(f"  GPU Memory: {memory_info.get('gpu_memory_total_gb', 0):.1f} GB")
print(f"  System Memory: {memory_info['system_memory_total_gb']:.1f} GB")

print(f"\n🔧 Software Configuration:")
print(f"  PyTorch: {torch.__version__}")
print(f"  ControlNet Available: {CONFIG['environment']['controlnet_available']}")
print(f"  MiDaS Model: {CONFIG['models']['midas']['default']}")

print(f"\n📁 Directory Structure:")
print(f"  Data Path: {CONFIG['paths']['data_path']}")
print(f"  Models Path: {CONFIG['paths']['models_path']}")
print(f"  Checkpoints Path: {CONFIG['paths']['checkpoints_path']}")

print(f"\n📊 Processing Configuration:")
print(f"  Batch Size: {CONFIG['processing']['batch_size']}")
print(f"  Image Size: {CONFIG['processing']['image_size']}")
print(f"  Max Images per Session: {CONFIG['processing']['max_images_per_session']}")

print(f"\n✅ Ready for next notebook: 02_Data_Import_and_Preprocessing.ipynb")
print("=" * 60)

# Log completion
logger.info("Setup and installation completed successfully")
logger.info(f"System ready for pipeline execution")

# Save setup completion checkpoint
setup_completion_data = {
    'setup_completed': True,
    'config': CONFIG,
    'memory_info': memory_info,
    'completion_time': datetime.now().isoformat()
}

save_checkpoint(setup_completion_data, 'setup_completed', 'Setup and installation completed')

🔍 Performing final setup verification...



2026-01-09 16:30:05 - SwellSight - INFO - Setup and installation completed successfully
INFO:SwellSight:Setup and installation completed successfully
2026-01-09 16:30:05 - SwellSight - INFO - System ready for pipeline execution
INFO:SwellSight:System ready for pipeline execution


📋 Setup Verification Results:
Environment         : ✅ Ready
GPU                 : ✅ Tesla T4
Core Dependencies   : ✅ Installed
MiDaS               : ✅ Ready
ControlNet          : ✅ Ready
SwellSight Package  : ⚠️  Import issues (may be normal)
Directory Structure : ✅ Created
Configuration       : ✅ Saved

🎉 SETUP COMPLETED SUCCESSFULLY!

✅ Ready to proceed to the next notebook:
   📓 02_Data_Import_and_Preprocessing.ipynb

⚠️  Note: 1 components have warnings but should work

📚 Next Steps:
   1. Place real beach images in data/real/images/
   2. Run 02_Data_Import_and_Preprocessing.ipynb
   3. Execute the full pipeline with 03_MiDaS_Depth_Extraction.ipynb


SWELLSIGHT SETUP COMPLETED SUCCESSFULLY

📋 Session Information:
  Session ID: swellsight_20260109_162959
  Platform: COLAB
  Created: 2026-01-09T16:29:59.012570

🖥️  Hardware Configuration:
  Device: cuda:0
  GPU Available: True
  GPU Count: 1
  GPU Memory: 14.7 GB
  System Memory: 51.0 GB

🔧 Software Configuration:
  PyTorch: 2.9.0+c

2026-01-09 16:30:05 - SwellSight - INFO - Checkpoint saved: setup_completed -> /content/drive/MyDrive/SwellSight/checkpoints/setup_completed.json
INFO:SwellSight:Checkpoint saved: setup_completed -> /content/drive/MyDrive/SwellSight/checkpoints/setup_completed.json


PosixPath('/content/drive/MyDrive/SwellSight/checkpoints/setup_completed.json')

---

## Next Steps

The setup is now complete! You can proceed to the next notebook:

**[02_Data_Import_and_Preprocessing.ipynb](02_Data_Import_and_Preprocessing.ipynb)**

This notebook will handle:
- Loading real beach camera images
- Data validation and quality checks
- Metadata extraction and organization
- Initial data exploration

### Important Notes:
- All configuration and paths are now set up in the `SWELLSIGHT_CONFIG` global variable
- Logging is configured and available via the `SWELLSIGHT_LOGGER` global variable
- Checkpoint system is ready for saving intermediate results
- GPU memory management utilities are available

### Troubleshooting:
- If you encounter memory issues, use `clear_gpu_memory()` function
- Check system status anytime with `get_memory_usage()`
- All session data is automatically saved to Google Drive (if mounted)
- Logs are saved to the logs directory for debugging

---